In [1]:
import pandas as pd
import geopandas as gpd
import fiona
import regex as re

In [2]:
fiona.listlayers('C:/Users/Compumar/Documents/Job/IGEO/2020 - Metrogas - Geocoder/TarifaSocial/Bases_Geo/Geocoding_SanVicente.gdb')

['Calles']

In [3]:
fiona.listlayers('C:/Users/Compumar/Documents/Job/IGEO/2020 - Metrogas - Geocoder/TarifaSocial/NoMatch.gdb')

['AlmiranteBrown',
 'Avellaneda',
 'Berazategui',
 'CABA',
 'EEcheverria',
 'Ezeiza',
 'Florencia',
 'Lanus',
 'LomasDeZamora',
 'PresidentePeron',
 'Quilmes',
 'SanVicente']

In [4]:
Calles = gpd.read_file('C:/Users/Compumar/Documents/Job/IGEO/2020 - Metrogas - Geocoder/TarifaSocial/Bases_Geo/Geocoding_SanVicente.gdb', driver='FileGDB', layer='Calles')

In [5]:
CallesNorm = gpd.read_file('C:/Users/Compumar/Documents/Job/IGEO/2020 - Metrogas - Geocoder/TarifaSocial/NoMatch.gdb', driver='FileGDB', layer='SanVicente')

In [6]:
Calles.head()

,NOMCALLE,NOMPAR,NOMLOC,NOMBARRIO,DESDDER,DESDIZQ,HASTDER,HASTIZQ,CPADER,CPAIZQ,TIPVIA,Shape_Length,geometry
0,SALTA,SAN VICENTE,ALEJANDRO KORN,,200.0,201.0,298.0,299.0,1865,1865,CALLE,124.940219,"MULTILINESTRING ((6374287.407 6127991.230, 637..."
1,SAN VICENTE,SAN VICENTE,ALEJANDRO KORN,,300.0,301.0,398.0,399.0,1865,1865,CALLE,100.051767,"MULTILINESTRING ((6374353.268 6127636.120, 637..."
2,SAN VICENTE,SAN VICENTE,ALEJANDRO KORN,,0.0,0.0,0.0,0.0,1865,1865,CALLE,32.607910,"MULTILINESTRING ((6374203.012 6127542.611, 637..."
3,SAN VICENTE,SAN VICENTE,ALEJANDRO KORN,,0.0,0.0,0.0,0.0,1865,1865,CALLE,116.392432,"MULTILINESTRING ((6374076.930 6127464.672, 637..."
4,SAN VICENTE,SAN VICENTE,ALEJANDRO KORN,,102.0,101.0,198.0,199.0,1865,1865,CALLE,113.909252,"MULTILINESTRING ((6374557.015 6127766.769, 637..."


In [7]:
CallesNorm.head()

,Id_Usuario,Usuario,Calle,Numero,Piso,Departamento,CodigoPostal,Localidad,PartidoMunicipio,Provincia,geometry
0,3.000022e+10,IRENA CWALINSKA,CAMINO REAL,2064,,,1864,Alejandro Korn,San Vicente,Buenos Aires,None
1,3.000556e+10,SUSANA GRACIELA MARILLA,COLOMBRES (ARENALES),17,,,1865,San Vicente,San Vicente,Buenos Aires,None
2,3.000494e+10,CARLOS KAPROILIAN,MATHEU,2090,,,1865,San Vicente,San Vicente,Buenos Aires,None
3,3.000090e+10,MIRTA GRACIELA KADELA,RUTA N 210 (H. YRIGOYEN),27646,,,1864,Alejandro Korn,San Vicente,Buenos Aires,None
4,3.000327e+10,FIDENCIA JULIAN,LAS HERAS,1702,,,1864,Alejandro Korn,San Vicente,Buenos Aires,None


In [8]:
# size of correct streets
len(Calles['NOMCALLE'].unique())

397

In [9]:
# size of name to check/correct
len(CallesNorm['Calle'].unique())

138

In [10]:
# Calles.to_file('C:/Users/Compumar/Documents/Job/IGEO/2020 - Metrogas - Geocoder/TarifaSocial/AvellanedaCalles.geojson', driver='GeoJSON')
# CallesNorm.to_file('C:/Users/Compumar/Documents/Job/IGEO/2020 - Metrogas - Geocoder/TarifaSocial/AlmBrownNoMatch.geojson', driver='GeoJSON')

# Reemplazo automático por búsqueda

In [11]:
# Create a new field with a standar null string value 
CallesNorm['CALLE_NEW'] = 'Nulo'
CallesNorm['CALLE_STATE'] = 'Nulo'

### Street's names with (

In [12]:
# Select all street with a parentesis "(", and get unique street names
# This street are the ones that changed their names, so it could appear in one or the other way
DobleNombres = CallesNorm[CallesNorm['Calle'].str.contains('\(')]['Calle'].unique()
DobleNombres

array(['COLOMBRES (ARENALES)', 'RUTA N 210 (H. YRIGOYEN)',
       '12 DE OCTUBRE (EX 17 DE OCTUBR', 'LIBERTAD (DEWEY)',
       'TRIUNVIRATO (R. WALSH)', 'PERON J. D., PTE.(BV.DE MAYO)',
       'SERRANO ( LORENZO MIGUEL)', 'MAFFIA PEDRO (EX CONSTITUCION)'],
      dtype=object)

In [13]:
# loop over this names and change the new field names to "to be checked"
for i in range(0, len(DobleNombres)):
    Nombre_Calle =DobleNombres[i]
    CallesNorm.loc[CallesNorm.Calle == Nombre_Calle, 'CALLE_NEW'] = 'Chequear'
    CallesNorm.loc[CallesNorm.Calle == Nombre_Calle, 'CALLE_STATE'] = 'Chequear'

### Street's names with "EX" word

In [14]:
# Select all street with a parentesis "EX", and get unique street names
# This street are the ones that changed their names, so it could appear in one or the other way
EXNombres = CallesNorm[CallesNorm['Calle'].str.contains('EX')]['Calle'].unique()
EXNombres

array(['12 DE OCTUBRE (EX 17 DE OCTUBR', 'MAFFIA PEDRO (EX CONSTITUCION)'],
      dtype=object)

In [15]:
for i in range(0, len(EXNombres)):
    Nombre_Calle = EXNombres[i]
    CallesNorm.loc[CallesNorm.Calle == Nombre_Calle, 'CALLE_NEW'] = 'Chequear'
    CallesNorm.loc[CallesNorm.Calle == Nombre_Calle, 'CALLE_STATE'] = 'Chequear'

In [16]:
len(CallesNorm.loc[CallesNorm['CALLE_NEW'] == 'Chequear', 'Calle'].unique())

8

### Loop names

#### Search exact name match

In [17]:
# Select the names of streets still not corrected
Nombres_Calles = CallesNorm.loc[CallesNorm.CALLE_NEW == 'Nulo', 'Calle'].unique()

# loop over them
for i in range(0, len(Nombres_Calles)):
    
    # Select a name
    Nombre_Calle = Nombres_Calles[i]
    Nombre_Calle_N = Nombres_Calles[i]
    
    # replace Ñ by N
    if(Nombre_Calle_N.find('Ñ')>=0):
        Nombre_Calle_N = Nombre_Calle_N.replace('Ñ', 'N')
            
    if(Nombre_Calle_N.find('-')>=0):
        Nombre_Calle_N = Nombre_Calle_N.split('-')
        Nombre_Calle_N = Nombre_Calle_N[-1]
        Nombre_Calle_N = Nombre_Calle_N.split()
        Nombre_Calle_N = ' '.join(Nombre_Calle_N)
    
    # search if the street's name is in the correct street layer
    if(len(Calles.loc[Calles.NOMCALLE ==  Nombre_Calle_N])!=0):
        b = [Nombre_Calle_N]
    # esle try to find the word in the name of the correct street layer
    else:
        b = Calles[Calles['NOMCALLE'].str.contains(Nombre_Calle_N)]['NOMCALLE'].unique()
        
    if(len(b)==1):   # if one coincidence, set the value to this name 
        CallesNorm.loc[CallesNorm.Calle == Nombre_Calle, 'CALLE_NEW'] = b[0]
        CallesNorm.loc[CallesNorm.Calle == Nombre_Calle, 'CALLE_STATE'] = 'Check'
    elif(len(b)>1):   # if more than one coincidence, set the value to 'many'
        CallesNorm.loc[CallesNorm.Calle == Nombre_Calle, 'CALLE_NEW'] = ' / '.join(list(b))
        CallesNorm.loc[CallesNorm.Calle == Nombre_Calle, 'CALLE_STATE'] = 'Varios'
    else:    # If no coincidence, set value to null string
        CallesNorm.loc[CallesNorm.Calle == Nombre_Calle, 'CALLE_NEW'] = 'Nulo'
        CallesNorm.loc[CallesNorm.Calle == Nombre_Calle, 'CALLE_STATE'] = 'Nulo'

In [18]:
print('Check:', len(CallesNorm.loc[CallesNorm['CALLE_STATE'] == 'Check', 'Calle'].unique()))
print('Nulls:', len(CallesNorm.loc[CallesNorm['CALLE_STATE'] == 'Nulo', 'Calle'].unique()))
print('Varios:', len(CallesNorm.loc[CallesNorm['CALLE_STATE'] == 'Varios', 'Calle'].unique()))

Check: 80
Nulls: 47
Varios: 3


#### Search exact name match, first word at the end

In [19]:
# Select the names of streets still not corrected
Nombres_Calles = CallesNorm.loc[CallesNorm.CALLE_NEW == 'Nulo', 'Calle'].unique()

# loop over them
for i in range(0, len(Nombres_Calles)):
    
    # Select a name
    Nombre_Calle = Nombres_Calles[i]
    
    x_calle = Nombre_Calle.replace(',', ' ')
    x_calle = x_calle.replace('.', ' ')
    # replace Ñ by N
    if(x_calle.find('Ñ')>=0):
        x_calle = x_calle.replace('Ñ', 'N')
        
    if(x_calle.find('-')>=0):
        x_calle = x_calle.split('-')
        x_calle = x_calle[-1]
        x_calle = x_calle.split()
        x_calle = ' '.join(x_calle)
        
    x_calle= x_calle.split()
    x_calle = ' '.join(x_calle[1:]) + ' ' + x_calle[0]
    
    # search if the street's name is in the correct street layer
    if(len(Calles.loc[Calles.NOMCALLE ==  x_calle])!=0):
        b = [x_calle]
    # esle try to find the word in the name of the correct street layer
    else:
        b = Calles[Calles['NOMCALLE'].str.contains(x_calle)]['NOMCALLE'].unique()
        
    if(len(b)==1):   # if one coincidence, set the value to this name 
        CallesNorm.loc[CallesNorm.Calle == Nombre_Calle, 'CALLE_NEW'] = b[0]
        CallesNorm.loc[CallesNorm.Calle == Nombre_Calle, 'CALLE_STATE'] = 'Check'
    elif(len(b)>1):   # if more than one coincidence, set the value to 'many'
        CallesNorm.loc[CallesNorm.Calle == Nombre_Calle, 'CALLE_NEW'] = ' / '.join(list(b))
        CallesNorm.loc[CallesNorm.Calle == Nombre_Calle, 'CALLE_STATE'] = 'Varios'
    else:    # If no coincidence, set value to null string
        CallesNorm.loc[CallesNorm.Calle == Nombre_Calle, 'CALLE_NEW'] = 'Nulo'
        CallesNorm.loc[CallesNorm.Calle == Nombre_Calle, 'CALLE_STATE'] = 'Nulo'

In [20]:
print('Check:', len(CallesNorm.loc[CallesNorm['CALLE_STATE'] == 'Check', 'Calle'].unique()))
print('Nulls:', len(CallesNorm.loc[CallesNorm['CALLE_STATE'] == 'Nulo', 'Calle'].unique()))
print('Varios:', len(CallesNorm.loc[CallesNorm['CALLE_STATE'] == 'Varios', 'Calle'].unique()))

Check: 91
Nulls: 36
Varios: 3


#### Search for splited names match - all words of the name coincidence ('PCIA. BS. AS.' => 'PCIA' + 'BS' + 'AS') 

In [21]:
# Select the names of streets still not corrected
Nombres_Calles = CallesNorm.loc[CallesNorm.CALLE_NEW == 'Nulo', 'Calle'].unique()

# loop over them
for i in range(0, len(Nombres_Calles)):
    Nombre_Calle = Nombres_Calles[i]
    
    # replace . and , by space
    Nombre_Calle_Split = Nombre_Calle.replace(',', ' ')
    Nombre_Calle_Split = Nombre_Calle_Split.replace('.', ' ')
    # replace Ñ by N
    if(Nombre_Calle_Split.find('Ñ')>=0):
        Nombre_Calle_Split = Nombre_Calle_Split.replace('Ñ', 'N')
    
    # delete number to take name ('15 - San Martin' => 'San Martin')
    if(Nombre_Calle_Split.find('-')>=0):
        Nombre_Calle_Split = Nombre_Calle_Split.split('-')
        Nombre_Calle_Split = Nombre_Calle_Split[-1]
        Nombre_Calle_Split = Nombre_Calle_Split.split()
        Nombre_Calle_Split = ' '.join(Nombre_Calle_Split)
    
    Nombre_Calle_Split = Nombre_Calle_Split.split()
    
    # if the name has two word (usually names)
    if (len(Nombre_Calle_Split)==2):
        b = Calles[(Calles['NOMCALLE'].str.contains(Nombre_Calle_Split[0])) & (Calles['NOMCALLE'].str.contains(Nombre_Calle_Split[1]))]['NOMCALLE'].unique()
    elif (len(Nombre_Calle_Split)==3):
        b = Calles[(Calles['NOMCALLE'].str.contains(Nombre_Calle_Split[0])) & (Calles['NOMCALLE'].str.contains(Nombre_Calle_Split[1])) & (Calles['NOMCALLE'].str.contains(Nombre_Calle_Split[2]))]['NOMCALLE'].unique()
    elif (len(Nombre_Calle_Split)==4):
        b = Calles[(Calles['NOMCALLE'].str.contains(Nombre_Calle_Split[0])) & (Calles['NOMCALLE'].str.contains(Nombre_Calle_Split[1])) & (Calles['NOMCALLE'].str.contains(Nombre_Calle_Split[2])) & (Calles['NOMCALLE'].str.contains(Nombre_Calle_Split[3]))]['NOMCALLE'].unique()
    
    if ((len(Nombre_Calle_Split)>=2) & (len(Nombre_Calle_Split)<=4)):
        if(len(b)==1):   # if one coincidence, set the value to this name 
            CallesNorm.loc[CallesNorm.Calle == Nombre_Calle, 'CALLE_NEW'] = b[0]
            CallesNorm.loc[CallesNorm.Calle == Nombre_Calle, 'CALLE_STATE'] = 'Check'
        elif(len(b)>1):   # if more than one coincidence, set the value to 'many'
            CallesNorm.loc[CallesNorm.Calle == Nombre_Calle, 'CALLE_NEW'] = ' / '.join(list(b))
            CallesNorm.loc[CallesNorm.Calle == Nombre_Calle, 'CALLE_STATE'] = 'Varios'
        else:    # If no coincidence, set value to null string
            CallesNorm.loc[CallesNorm.Calle == Nombre_Calle, 'CALLE_NEW'] = 'Nulo'
            CallesNorm.loc[CallesNorm.Calle == Nombre_Calle, 'CALLE_STATE'] = 'Nulo'

In [22]:
print('Check:', len(CallesNorm.loc[CallesNorm['CALLE_STATE'] == 'Check', 'Calle'].unique()))
print('Nulls:', len(CallesNorm.loc[CallesNorm['CALLE_STATE'] == 'Nulo', 'Calle'].unique()))
print('Varios:', len(CallesNorm.loc[CallesNorm['CALLE_STATE'] == 'Varios', 'Calle'].unique()))

Check: 102
Nulls: 24
Varios: 4


#### Search for inverted names match - two words (invert name and surname)

In [23]:
# Select the names of streets still not corrected
Nombres_Calles = CallesNorm.loc[CallesNorm.CALLE_NEW == 'Nulo', 'Calle'].unique()

# loop over them
for i in range(0, len(Nombres_Calles)):
    Nombre_Calle = Nombres_Calles[i]
    
    # Delete comas and dots
    Nombre_Calle_Inv = Nombre_Calle.replace(',', ' ')
    Nombre_Calle_Inv = Nombre_Calle_Inv.replace('.', ' ')
    # change Ñ by N
    if(Nombre_Calle_Inv.find('Ñ')>=0):
        Nombre_Calle_Inv = Nombre_Calle_Inv.replace('Ñ', 'N')
        
    # delete number to take name ('15 - San Martin' => 'San Martin')
    if(Nombre_Calle_Inv.find('-')>=0):
        Nombre_Calle_Inv = Nombre_Calle_Inv.split('-')
        Nombre_Calle_Inv = Nombre_Calle_Inv[-1]
        Nombre_Calle_Inv = Nombre_Calle_Inv.split()
        Nombre_Calle_Inv = ' '.join(Nombre_Calle_Inv)
    
    # if the name has two word (usually names)
    if (len(Nombre_Calle_Inv.split())==2):
        # Invert the name ('OLARRA JOSE' => 'JOSE OLARRA')
        Nombre_Calle_Inv= Nombre_Calle_Inv.split()[1] + ' ' + Nombre_Calle_Inv.split()[0]
        
        # Repeat previos process
        # search if the street's name is in the correct street layer
        if(len(Calles.loc[Calles.NOMCALLE ==  Nombre_Calle_Inv])!=0):
            b = [Nombre_Calle]
        # esle try to find the word in the name of the correct street layer
        else:
            b = Calles[Calles['NOMCALLE'].str.contains(Nombre_Calle_Inv)]['NOMCALLE'].unique()

        if(len(b)==1):   # if one coincidence, set the value to this name 
            CallesNorm.loc[CallesNorm.Calle == Nombre_Calle, 'CALLE_NEW'] = b[0]
            CallesNorm.loc[CallesNorm.Calle == Nombre_Calle, 'CALLE_STATE'] = 'Check'
        elif(len(b)>1):   # if more than one coincidence, set the value to 'many'
            CallesNorm.loc[CallesNorm.Calle == Nombre_Calle, 'CALLE_NEW'] = ' / '.join(list(b))
            CallesNorm.loc[CallesNorm.Calle == Nombre_Calle, 'CALLE_STATE'] = 'Varios'
        else:    # If no coincidence, set value to null string
            CallesNorm.loc[CallesNorm.Calle == Nombre_Calle, 'CALLE_NEW'] = 'Nulo'
            CallesNorm.loc[CallesNorm.Calle == Nombre_Calle, 'CALLE_STATE'] = 'Nulo'

In [24]:
print('Check:', len(CallesNorm.loc[CallesNorm['CALLE_STATE'] == 'Check', 'Calle'].unique()))
print('Nulls:', len(CallesNorm.loc[CallesNorm['CALLE_STATE'] == 'Nulo', 'Calle'].unique()))
print('Varios:', len(CallesNorm.loc[CallesNorm['CALLE_STATE'] == 'Varios', 'Calle'].unique()))

Check: 102
Nulls: 24
Varios: 4


#### Search for two longest words in the names match 

In [25]:
# Select the names of streets still not corrected
Nombres_Calles = CallesNorm.loc[CallesNorm.CALLE_NEW == 'Nulo', 'Calle'].unique()

# loop over them
for i in range(0, len(Nombres_Calles)):
    Nombre_Calle = Nombres_Calles[i]
    
     # change Ñ by N
    Nombre_Calle_FT = Nombre_Calle.replace(',', ' ')
    Nombre_Calle_FT = Nombre_Calle_FT.replace('.', ' ') 
    if(Nombre_Calle_FT.find('Ñ')>=0):
        Nombre_Calle_FT = Nombre_Calle.replace('Ñ', 'N')
        
    # delete number to take name ('15 - San Martin' => 'San Martin')
    if(Nombre_Calle_FT.find('-')>=0):
        Nombre_Calle_FT = Nombre_Calle_FT.split('-')
        Nombre_Calle_FT = Nombre_Calle_FT[-1]
        Nombre_Calle_FT = Nombre_Calle_FT.split()
        Nombre_Calle_FT = ' '.join(Nombre_Calle_FT)
        
    # Take the first word of the name
    Nombre_Calle_FT = Nombre_Calle_FT.split()
    Nombre_Calle_FT.sort(key=len, reverse=True)
    
    # If name has more than three letters
    if(len(Nombre_Calle_FT) >= 2):
        Nombre_Calle_FT = Nombre_Calle_FT[0:2]
        b = Calles[(Calles['NOMCALLE'].str.contains(Nombre_Calle_FT[0])) & (Calles['NOMCALLE'].str.contains(Nombre_Calle_FT[1]))]['NOMCALLE'].unique()

        if(len(b)==1):   # if one coincidence, set the value to this name 
            CallesNorm.loc[CallesNorm.Calle == Nombre_Calle, 'CALLE_NEW'] = b[0]
            CallesNorm.loc[CallesNorm.Calle == Nombre_Calle, 'CALLE_STATE'] = 'Check'
        elif(len(b)>1):   # if more than one coincidence, set the value to 'many'
            CallesNorm.loc[CallesNorm.Calle == Nombre_Calle, 'CALLE_NEW'] = ' / '.join(list(b))
            CallesNorm.loc[CallesNorm.Calle == Nombre_Calle, 'CALLE_STATE'] = 'Varios'
        else:    # If no coincidence, set value to null string
            CallesNorm.loc[CallesNorm.Calle == Nombre_Calle, 'CALLE_NEW'] = 'Nulo'
            CallesNorm.loc[CallesNorm.Calle == Nombre_Calle, 'CALLE_STATE'] = 'Nulo'

In [26]:
print('Check:', len(CallesNorm.loc[CallesNorm['CALLE_STATE'] == 'Check', 'Calle'].unique()))
print('Nulls:', len(CallesNorm.loc[CallesNorm['CALLE_STATE'] == 'Nulo', 'Calle'].unique()))
print('Varios:', len(CallesNorm.loc[CallesNorm['CALLE_STATE'] == 'Varios', 'Calle'].unique()))

Check: 102
Nulls: 23
Varios: 5


#### Search for first word names match  (longer than three letters)

In [27]:
# Select the names of streets still not corrected
Nombres_Calles = CallesNorm.loc[CallesNorm.CALLE_NEW == 'Nulo', 'Calle'].unique()

# loop over them
for i in range(0, len(Nombres_Calles)):
    Nombre_Calle = Nombres_Calles[i]
    
    Nombre_Calle_Inv = Nombre_Calle.replace(',', ' ')
    Nombre_Calle_Inv = Nombre_Calle_Inv.replace('.', ' ')
     # change Ñ by N
    if(Nombre_Calle_Inv.find('Ñ')>=0):
        Nombre_Calle_Inv = Nombre_Calle_Inv.replace('Ñ', 'N')
        
    if(Nombre_Calle_Inv.find('-')>=0):
        Nombre_Calle_Inv = Nombre_Calle_Inv.split('-')
        Nombre_Calle_Inv = Nombre_Calle_Inv[-1]
        Nombre_Calle_Inv = Nombre_Calle_Inv.split()
        Nombre_Calle_Inv = ' '.join(Nombre_Calle_Inv)
        
    # Take the first word of the name
    Nombre_Calle_Inv = Nombre_Calle.split()[0]
    
    # If name has more than three letters
    if(len(Nombre_Calle_Inv) >3):
        b = Calles[Calles['NOMCALLE'].str.contains(Nombre_Calle_Inv)]['NOMCALLE'].unique()

        if(len(b)==1):   # if one coincidence, set the value to this name 
            CallesNorm.loc[CallesNorm.Calle == Nombre_Calle, 'CALLE_NEW'] = b[0]
            CallesNorm.loc[CallesNorm.Calle == Nombre_Calle, 'CALLE_STATE'] = 'Check'
        elif(len(b)>1):   # if more than one coincidence, set the value to 'many'
            CallesNorm.loc[CallesNorm.Calle == Nombre_Calle, 'CALLE_NEW'] = ' / '.join(list(b))
            CallesNorm.loc[CallesNorm.Calle == Nombre_Calle, 'CALLE_STATE'] = 'Varios'
        else:    # If no coincidence, set value to null string
            CallesNorm.loc[CallesNorm.Calle == Nombre_Calle, 'CALLE_NEW'] = 'Nulo'
            CallesNorm.loc[CallesNorm.Calle == Nombre_Calle, 'CALLE_STATE'] = 'Nulo'

In [28]:
print('Check:', len(CallesNorm.loc[CallesNorm['CALLE_STATE'] == 'Check', 'Calle'].unique()))
print('Nulls:', len(CallesNorm.loc[CallesNorm['CALLE_STATE'] == 'Nulo', 'Calle'].unique()))
print('Varios:', len(CallesNorm.loc[CallesNorm['CALLE_STATE'] == 'Varios', 'Calle'].unique()))

Check: 107
Nulls: 15
Varios: 8


#### Search for the longest words in the names match 

In [29]:
# Select the names of streets still not corrected
Nombres_Calles = CallesNorm.loc[CallesNorm.CALLE_NEW == 'Nulo', 'Calle'].unique()

# loop over them
for i in range(0, len(Nombres_Calles)):
    Nombre_Calle = Nombres_Calles[i]
    
     # change Ñ by N
    Nombre_Calle_FT = Nombre_Calle.replace(',', ' ')
    Nombre_Calle_FT = Nombre_Calle_FT.replace('.', ' ') 
    if(Nombre_Calle_FT.find('Ñ')>=0):
        Nombre_Calle_FT = Nombre_Calle.replace('Ñ', 'N')
        
    if(Nombre_Calle_FT.find('-')>=0):
        Nombre_Calle_FT = Nombre_Calle_FT.split('-')
        Nombre_Calle_FT = Nombre_Calle_FT[-1]
        Nombre_Calle_FT = Nombre_Calle_FT.split()
        Nombre_Calle_FT = ' '.join(Nombre_Calle_FT)
        
    # Take the first word of the name
    Nombre_Calle_FT = Nombre_Calle_FT.split()
    Nombre_Calle_FT.sort(key=len, reverse=True)
    
    # If name has more than three letters
    if(len(Nombre_Calle_FT) >= 2):
        Nombre_Calle_FT = Nombre_Calle_FT[0:1]
        b = Calles[(Calles['NOMCALLE'].str.contains(Nombre_Calle_FT[0]))]['NOMCALLE'].unique()

        if(len(b)==1):   # if one coincidence, set the value to this name 
            CallesNorm.loc[CallesNorm.Calle == Nombre_Calle, 'CALLE_NEW'] = b[0]
            CallesNorm.loc[CallesNorm.Calle == Nombre_Calle, 'CALLE_STATE'] = 'Check'
        elif(len(b)>1):   # if more than one coincidence, set the value to 'many'
            CallesNorm.loc[CallesNorm.Calle == Nombre_Calle, 'CALLE_NEW'] = ' / '.join(list(b))
            CallesNorm.loc[CallesNorm.Calle == Nombre_Calle, 'CALLE_STATE'] = 'Varios'
        else:    # If no coincidence, set value to null string
            CallesNorm.loc[CallesNorm.Calle == Nombre_Calle, 'CALLE_NEW'] = 'Nulo'
            CallesNorm.loc[CallesNorm.Calle == Nombre_Calle, 'CALLE_STATE'] = 'Nulo'

In [30]:
print('Check:', len(CallesNorm.loc[CallesNorm['CALLE_STATE'] == 'Check', 'Calle'].unique()))
print('Nulls:', len(CallesNorm.loc[CallesNorm['CALLE_STATE'] == 'Nulo', 'Calle'].unique()))
print('Varios:', len(CallesNorm.loc[CallesNorm['CALLE_STATE'] == 'Varios', 'Calle'].unique()))

Check: 110
Nulls: 11
Varios: 9


#### Search for second word names match

In [31]:
# # Select the names of streets still not corrected
# Nombres_Calles = CallesNorm.loc[CallesNorm.CALLE_NEW == 'Nulo', 'Calle'].unique()

# # loop over them
# for i in range(0, len(Nombres_Calles)):
#     Nombre_Calle = Nombres_Calles[i]
    
#      # change Ñ by N
#     if(Nombre_Calle.find('Ñ')>=0):
#         Nombre_Calle = Nombre_Calle.replace('Ñ', 'N')
#     Nombre_Calle_Inv = Nombre_Calle_Inv.replace(',', ' ')
#     Nombre_Calle_Inv = Nombre_Calle_Inv.replace('.', ' ')
    
#     if (len(Nombre_Calle.split())>=2):
#         Nombre_Calle_Inv = Nombre_Calle.split()[1]

#         if(len(Nombre_Calle_Inv) >3):
#             b = Calles[Calles['NOMCALLE'].str.contains(Nombre_Calle_Inv)]['NOMCALLE'].unique()

#             if(len(b)==1):   # if one coincidence, set the value to this name 
#                 CallesNorm.loc[CallesNorm.Calle == Nombre_Calle, 'CALLE_NEW'] = b[0]
#                 CallesNorm.loc[CallesNorm.Calle == Nombre_Calle, 'CALLE_STATE'] = 'Check'
#             elif(len(b)>1):   # if more than one coincidence, set the value to 'many'
#                 CallesNorm.loc[CallesNorm.Calle == Nombre_Calle, 'CALLE_NEW'] = ' / '.join(list(b))
#                 CallesNorm.loc[CallesNorm.Calle == Nombre_Calle, 'CALLE_STATE'] = 'Varios'
#             else:    # If no coincidence, set value to null string
#                 CallesNorm.loc[CallesNorm.Calle == Nombre_Calle, 'CALLE_NEW'] = 'Nulo'
#                 CallesNorm.loc[CallesNorm.Calle == Nombre_Calle, 'CALLE_STATE'] = 'Nulo'
#         else:
#             CallesNorm.loc[CallesNorm.Calle == Nombre_Calle, 'CALLE_NEW'] = 'Nulo'
#             CallesNorm.loc[CallesNorm.Calle == Nombre_Calle, 'CALLE_STATE'] = 'Nulo'
#     else:
#         CallesNorm.loc[CallesNorm.Calle == Nombre_Calle, 'CALLE_NEW'] = 'Nulo'
#         CallesNorm.loc[CallesNorm.Calle == Nombre_Calle, 'CALLE_STATE'] = 'Nulo'

In [32]:
# print('Check:', len(CallesNorm.loc[CallesNorm['CALLE_STATE'] == 'Check', 'Calle'].unique()))
# print('Nulls:', len(CallesNorm.loc[CallesNorm['CALLE_STATE'] == 'Nulo', 'Calle'].unique()))
# print('Varios:', len(CallesNorm.loc[CallesNorm['CALLE_STATE'] == 'Varios', 'Calle'].unique()))

# Search for street's number names ('11' | '11 A')

In [33]:
# Select the names of streets still not corrected
Nombres_Calles = CallesNorm.loc[CallesNorm.CALLE_NEW == 'Nulo', 'Calle'].unique()

# loop over them
for i in range(0, len(Nombres_Calles)):
    Nombre_Calle = Nombres_Calles[i]
    Nombre_Calle_FT = Nombres_Calles[i]
    
    # take the number part
    Nombre_Calle_FT = Nombre_Calle_FT.split()
    Nombre_Calle_FT = Nombre_Calle_FT[0]
    Nombre_Calle_FT = Nombre_Calle_FT.replace(' ', '')
    
    if ( Nombre_Calle_FT[0] == 'C' ):
        Nombre_Calle_FT = Nombre_Calle_FT.replace('C', '')
    
    if (len(Nombre_Calle_FT)<6):
        CallesNumeros = re.findall('\d+', Nombre_Calle_FT )
        
        if( len(CallesNumeros) > 0 ):
            CallesNumeros = CallesNumeros[0]
        
            if ( len(Nombre_Calle_FT) == len(CallesNumeros) ):
                Nombre_Calle_FT = CallesNumeros
            elif ( len(Nombre_Calle_FT) == len(CallesNumeros) + 1 ):
                c = Nombre_Calle_FT[len(Nombre_Calle_FT)-1]
                Nombre_Calle_FT = CallesNumeros+''+c

            if(len(Calles.loc[Calles.NOMCALLE ==  Nombre_Calle_FT])!=0):
                b = [Nombre_Calle_FT]
            else:
                b = Calles[Calles['NOMCALLE'].str.contains(Nombre_Calle_FT)]['NOMCALLE'].unique()

            if(len(b)==1):   # if one coincidence, set the value to this name 
                CallesNorm.loc[CallesNorm.Calle == Nombre_Calle, 'CALLE_NEW'] = b[0]
                CallesNorm.loc[CallesNorm.Calle == Nombre_Calle, 'CALLE_STATE'] = 'Check'
            elif(len(b)>1):   # if more than one coincidence, set the value to 'many'
                CallesNorm.loc[CallesNorm.Calle == Nombre_Calle, 'CALLE_NEW'] = ' / '.join(list(b))
                CallesNorm.loc[CallesNorm.Calle == Nombre_Calle, 'CALLE_STATE'] = 'Varios'
            else:    # If no coincidence, set value to null string
                CallesNorm.loc[CallesNorm.Calle == Nombre_Calle, 'CALLE_NEW'] = 'Nulo'
                CallesNorm.loc[CallesNorm.Calle == Nombre_Calle, 'CALLE_STATE'] = 'Nulo'

In [34]:
print('Check:', len(CallesNorm.loc[CallesNorm['CALLE_STATE'] == 'Check', 'Calle'].unique()))
print('Nulls:', len(CallesNorm.loc[CallesNorm['CALLE_STATE'] == 'Nulo', 'Calle'].unique()))
print('Varios:', len(CallesNorm.loc[CallesNorm['CALLE_STATE'] == 'Varios', 'Calle'].unique()))

Check: 110
Nulls: 11
Varios: 9


# Errors

In [35]:
CallesNorm.loc[CallesNorm['CALLE_STATE'] == 'Chequear', 'Calle'].unique()

array(['COLOMBRES (ARENALES)', 'RUTA N 210 (H. YRIGOYEN)',
       '12 DE OCTUBRE (EX 17 DE OCTUBR', 'LIBERTAD (DEWEY)',
       'TRIUNVIRATO (R. WALSH)', 'PERON J. D., PTE.(BV.DE MAYO)',
       'SERRANO ( LORENZO MIGUEL)', 'MAFFIA PEDRO (EX CONSTITUCION)'],
      dtype=object)

In [36]:
CallesNorm.loc[CallesNorm['CALLE_STATE'] == 'Nulo', 'Calle'].unique()

array(['NECOCHEA', 'DEL PROGRESO', 'RAPALLINI', 'JAPON', 'O´HIGGINS',
       'VITALLER', 'ALVARADO', 'FELS T.', 'GARAT', 'CUITIÑO MIGUEL',
       'PARRAVICCINI F.'], dtype=object)

In [37]:
CallesNorm.loc[CallesNorm['CALLE_STATE'] == 'Varios', 'Calle'].unique()

array(['ISABEL LA CATOLICA', '1 DE MAYO', 'QUIROGA', 'MITRE',
       'AVELLANEDA', 'BV.DE MAYO', 'QUIROGA G.', 'PERON J. D., PTE.',
       'OBLIGADO RAFAEL'], dtype=object)

In [38]:
CallesNorm.Numero = pd.to_numeric(CallesNorm.Numero, errors='coerce').astype('Int64')

# Correction - Street´s Names to Check

In [64]:
x = x+1
print(x+1, '/', len(CallesNorm.loc[CallesNorm['CALLE_STATE'] == 'Chequear', 'Calle'].unique()))
CallesNorm.loc[CallesNorm['CALLE_STATE'] == 'Chequear', 'Calle'].unique()[x]

8 / 8


'MAFFIA PEDRO (EX CONSTITUCION)'

In [67]:
Calles[Calles['NOMCALLE'].str.contains('PEDRO')]['NOMCALLE'].unique()

array(['PEDRO QUIROGA', 'SAN PEDRO'], dtype=object)

In [60]:
print(Calles.loc[Calles.NOMCALLE == 'AVENIDA PRESIDENTE JUAN DOMINGO PERON', 'DESDDER'].unique())
print(Calles.loc[Calles.NOMCALLE == 'BOULEVAR DE MAYO', 'DESDDER'].unique())

[   0.  900.  700. 1350.  800. 1000.  400.  500. 1700. 1750. 1500.  600.
 2700. 2300. 1400. 1100.]
[400.   2. 200. 600. 102. 300. 500.]


In [68]:
CallesNorm.loc[(CallesNorm.Calle == 'COLOMBRES (ARENALES)'), 'CALLE_NEW'] = 'COLOMBRES'
CallesNorm.loc[(CallesNorm.Calle == 'RUTA N 210 (H. YRIGOYEN)'), 'CALLE_NEW'] = 'RUTA PROVINCIAL 210'
CallesNorm.loc[(CallesNorm.Calle == '12 DE OCTUBRE (EX 17 DE OCTUBR'), 'CALLE_NEW'] = '12 DE OCTUBRE'
CallesNorm.loc[(CallesNorm.Calle == 'LIBERTAD (DEWEY)'), 'CALLE_NEW'] = 'LIBERTAD'
CallesNorm.loc[(CallesNorm.Calle == 'TRIUNVIRATO (R. WALSH)'), 'CALLE_NEW'] = 'RODOLFO WALSH'
CallesNorm.loc[(CallesNorm.Calle == 'PERON J. D., PTE.(BV.DE MAYO)'), 'CALLE_NEW'] = 'AVENIDA PRESIDENTE JUAN DOMINGO PERON'
CallesNorm.loc[(CallesNorm.Calle == 'SERRANO ( LORENZO MIGUEL)'), 'CALLE_NEW'] = 'SERRANO'
CallesNorm.loc[(CallesNorm.Calle == 'MAFFIA PEDRO (EX CONSTITUCION)'), 'CALLE_NEW'] = 'MAFFIA PEDRO'

# Correction - Null Street´s Names

In [90]:
x = x+1
print(x+1, '/', len(CallesNorm.loc[CallesNorm['CALLE_STATE'] == 'Nulo', 'Calle'].unique()))
CallesNorm.loc[CallesNorm['CALLE_STATE'] == 'Nulo', 'Calle'].unique()[x]

11 / 11


'PARRAVICCINI F.'

In [91]:
Calles[Calles['NOMCALLE'].str.contains('PAR')]['NOMCALLE'].unique()

array(['LOS PARAISOS', 'FLORENCIO PARRAVICINI', 'PARIS', 'DEL PARNASO',
       'MIGUEL ANGEL PARDO'], dtype=object)

In [92]:
CallesNorm.loc[(CallesNorm.Calle == 'O´HIGGINS'), 'CALLE_NEW'] = 'BERNARDO O HIGGINS'
CallesNorm.loc[(CallesNorm.Calle == 'FELS T.'), 'CALLE_NEW'] = 'TEODORO FELD'
CallesNorm.loc[(CallesNorm.Calle == 'GARAT'), 'CALLE_NEW'] = 'JUAN DE GARAY'
CallesNorm.loc[(CallesNorm.Calle == 'CUITIÑO MIGUEL'), 'CALLE_NEW'] = 'MIGUEL CUTINO'
CallesNorm.loc[(CallesNorm.Calle == 'PARRAVICCINI F.'), 'CALLE_NEW'] = 'FLORENCIO PARRAVICINI'

# Correction - Street´s Names to Check

In [102]:
x = x+1
print(x+1, '/', len(CallesNorm.loc[CallesNorm['CALLE_STATE'] == 'Varios', 'Calle'].unique()))
NombreCalle = CallesNorm.loc[CallesNorm['CALLE_STATE'] == 'Varios', 'Calle'].unique()[x]
print(NombreCalle, '=> Opciones:', CallesNorm.loc[CallesNorm.Calle == NombreCalle, 'CALLE_NEW'].unique())

9 / 9
OBLIGADO RAFAEL => Opciones: ['OBLIGADO / VUELTA DE OBLIGADO']


In [162]:
Calles[Calles['NOMCALLE'].str.contains('407')]['NOMCALLE'].unique()

array([], dtype=object)

In [143]:
print(Calles.loc[Calles.NOMCALLE == 'GUIDO SPANO', 'DESDDER'].unique())
print(Calles.loc[Calles.NOMCALLE == 'GUIDO Y SPANO', 'DESDDER'].unique())

[600.]
[0.00e+00 2.30e+03 1.15e+03 1.00e+03 2.00e+02 2.40e+03 2.00e+00]


In [103]:
CallesNorm.loc[(CallesNorm.Calle == 'ISABEL LA CATOLICA'), 'CALLE_NEW'] = 'ISABEL LA CATOLOCA'
CallesNorm.loc[(CallesNorm.Calle == 'BV.DE MAYO'), 'CALLE_NEW'] = 'BOULEVAR DE MAYO'
CallesNorm.loc[(CallesNorm.Calle == 'PERON J. D., PTE.'), 'CALLE_NEW'] = 'AVENIDA PRESIDENTE JUAN DOMINGO PERON'
CallesNorm.loc[(CallesNorm.Calle == 'OBLIGADO RAFAEL'), 'CALLE_NEW'] = 'OBLIGADO'

# Export File

In [104]:
CallesNorm.to_csv('C:/Users/Compumar/Documents/Job/IGEO/2020 - Metrogas - Geocoder/TarifaSocial/SANVICENTE_nomatch.csv')

# Street Check

In [105]:
CallesCheck = pd.DataFrame(CallesNorm.Calle.unique())
CallesCheck

,0
0,CAMINO REAL
1,COLOMBRES (ARENALES)
2,MATHEU
3,RUTA N 210 (H. YRIGOYEN)
4,LAS HERAS
...,...
133,DON QUIJOTE
134,SANTO DOMINGO
135,MAFFIA PEDRO (EX CONSTITUCION)
136,CUITIÑO MIGUEL


In [106]:
for i in range(0,len(CallesCheck)):
    CallesCheck.loc[CallesCheck[0] == CallesCheck[0][i], 'Asig'] = CallesNorm.loc[CallesNorm.Calle == CallesCheck[0][i], 'CALLE_NEW'].unique()[0]
CallesCheck

,0,Asig
0,CAMINO REAL,CAMINO REAL
1,COLOMBRES (ARENALES),COLOMBRES
2,MATHEU,MATHEU
3,RUTA N 210 (H. YRIGOYEN),RUTA PROVINCIAL 210
4,LAS HERAS,LAS HERAS
...,...,...
133,DON QUIJOTE,DON QUIJOTE
134,SANTO DOMINGO,SANTO DOMINGO
135,MAFFIA PEDRO (EX CONSTITUCION),MAFFIA PEDRO
136,CUITIÑO MIGUEL,MIGUEL CUTINO


In [107]:
CallesCheck.to_csv('C:/Users/Compumar/Documents/Job/IGEO/2020 - Metrogas - Geocoder/TarifaSocial/SANVICENTE_nomatch_NamesCheck.csv')